In [ ]:
using LinearAlgebra
using Plots
using Ripserer
using Distances
using Plots
using DataFrames, CSV
using DelimitedFiles
using AbstractAlgebra
using MultivariateStats #for pca
using IterativeSolvers
using NearestNeighbors
include("mv_function_bens_alg.jl")


In [ ]:
#Globals
K = 10 # in KNN
#OG_K = 10
ITER = 30    # iterations of algorithm
break_count = -3 #iterations before breakingxs
min_loss = 1000
min_loss_2 = 1000
input_file ="./cool_big_data.csv"
output_file ="./point_color"

a_ = []
b_ = []
c_ = []
d_ = []
e_ = []
f_ = []

loss_ = []

In [ ]:
# Loss function 0.5 instead of pi

function loss_1(points, coord_ab, coord_cd, coord_ef)
    a_cd = mod(coord_cd[points[1]],1.0)
    b_cd = mod(coord_cd[points[2]],1.0)

    c_ab = mod(coord_ab[points[3]],1.0)
    d_ab = mod(coord_ab[points[4]],1.0)

    e_ab = mod(coord_ab[points[5]],1.0)
    f_ab = mod(coord_ab[points[6]],1.0)

    e_cd = mod(coord_cd[points[5]],1.0)
    f_cd = mod(coord_cd[points[6]],1.0)

    a_ef = mod(coord_ef[points[1]],1.0)
    b_ef = mod(coord_ef[points[2]],1.0)
    c_ef = mod(coord_ef[points[3]],1.0)
    d_ef = mod(coord_ef[points[4]],1.0)

    # ab_coords
    loss =(0.5 - mod(c_ab-d_ab, 1.0))^2 + (0.5 - mod(e_ab-f_ab,1.0))^2
    #loss += abs(0.25 - mod(c_ab - e_ab, 1.0))+ abs(0.25 - mod(e_ab - d_ab, 1.0)) + abs(0.25 - mod(d_ab - f_ab,1.0)) + + abs(0.25 - mod(f_ab - c_ab, 1.0))
    #cd_coords
    loss += (0.5 - mod(a_cd-b_cd, 1.0))^2 + (0.5 - mod(e_cd-f_cd,1.0))^2
    #loss += abs(0.25 - mod(a_cd - e_cd, 1.0)) + abs(0.25 - mod(e_cd - b_cd, 1.0)) + abs(0.25 - mod(b_cd - f_cd,1.0)) + abs(0.25 - mod(f_cd - a_cd, 1.0))
    #ef_coords
    loss += (0.5 - mod(a_ef-b_ef, 1.0))^2 + (0.5 - mod(c_ef-d_ef,1.0))^2
    #loss += abs(0.25 - mod(a_ef - c_ef, 1.0)) + abs(0.25 - mod(c_ef - b_ef, 1.0))  + abs(0.25 - mod(b_ef - d_ef,1.0)) + abs(0.25 - mod(d_ef - a_ef, 1.0)) 

    return loss
end

In [ ]:
# Loss function 0.5 instead of pi

function loss_2(points, coord_ab, coord_cd, coord_ef)
    a_cd = mod(coord_cd[points[1]],1.0)
    b_cd = mod(coord_cd[points[2]],1.0)

    c_ab = mod(coord_ab[points[3]],1.0)
    d_ab = mod(coord_ab[points[4]],1.0)

    e_ab = mod(coord_ab[points[5]],1.0)
    f_ab = mod(coord_ab[points[6]],1.0)

    e_cd = mod(coord_cd[points[5]],1.0)
    f_cd = mod(coord_cd[points[6]],1.0)

    a_ef = mod(coord_ef[points[1]],1.0)
    b_ef = mod(coord_ef[points[2]],1.0)
    c_ef = mod(coord_ef[points[3]],1.0)
    d_ef = mod(coord_ef[points[4]],1.0)

    # ab_coords
    #loss = abs(0.5 - mod(c_ab-d_ab, 1.0)) + abs(0.5 - mod(e_ab-f_ab,1.0))
    loss = abs(0.25 - mod(c_ab - e_ab, 1.0))+ abs(0.25 - mod(e_ab - d_ab, 1.0)) + abs(0.25 - mod(d_ab - f_ab,1.0)) + + abs(0.25 - mod(f_ab - c_ab, 1.0))
    #cd_coords
    #loss += abs(0.5 - mod(a_cd-b_cd, 1.0)) + abs(0.5 - mod(e_cd-f_cd,1.0))
    loss += abs(0.25 - mod(a_cd - e_cd, 1.0)) + abs(0.25 - mod(e_cd - b_cd, 1.0)) + abs(0.25 - mod(b_cd - f_cd,1.0)) + abs(0.25 - mod(f_cd - a_cd, 1.0))
    #ef_coords
    #loss += abs(0.5 - mod(a_ef-b_ef, 1.0)) + abs(0.5 - mod(c_ef-d_ef,1.0))
    loss += abs(0.25 - mod(a_ef - c_ef, 1.0)) + abs(0.25 - mod(c_ef - b_ef, 1.0))  + abs(0.25 - mod(b_ef - d_ef,1.0)) + abs(0.25 - mod(d_ef - a_ef, 1.0)) 

    return loss
end

In [ ]:
# Load Data
matrix = readdlm(input_file, ',', Float64)
data = []
for row in 1:length(matrix[:,1])
    arr = [matrix[row,i] for i in 1:length(matrix[1,:])]
    push!(data, arr)
end

# Do zeroth iteration of algerithm with random points

# Random distinct start points,
# Could be issues with distinct points yielding same piercing hole, and therefore cant do MV
a_0,b_0,c_0,d_0,e_0,f_0 = rand(1:15), rand(15:30), rand(30:45), rand(45:60), rand(60:75), rand(75:100)


# Store to see progress 

push!(a_, a_0)
push!(b_, b_0)
push!(c_, c_0)
push!(d_, d_0)
push!(e_, e_0)
push!(f_, f_0)


data_matrix = zeros(Float64, (3, length(data)))
    
for col in 1:length(data)
    data_matrix[:,col] = [data[col][1],data[col][2],data[col][3]]
end

kdtree = KDTree(data_matrix)





In [67]:
#min_loss_2  = 100   
for k in 1:ITER
    NN = length(a_)
    points = [a_[NN], b_[NN], c_[NN], d_[NN], e_[NN], f_[NN]]

    mv(points = points, start_points = true, input_file = input_file, output_file = output_file, double_penetration = true, tripple = true)
    # indexing from 2 because c
    matrix = readdlm(output_file * ".csv", ',')
    coord_ab = matrix[2:end,4]

    matrix = readdlm(output_file * "double" * ".csv", ',')
    coord_cd = matrix[2:end,4]

    matrix = readdlm(output_file * "tripple" * ".csv", ',')
    coord_ef = matrix[2:end,4]



    
    a_0 = points[1]
    b_0 = points[2]
    c_0 = points[3]
    d_0 = points[4]
    e_0 = points[5]
    f_0 = points[6]
    #println("Points ", points)

    l = length(a_)
    println("points")
    println("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," 
    * string(d_[l]-1) * "," * string(e_[l]-1) *  "," * string(f_[l]-1) * "]")
    println("-------")
    if k == 1
        min_loss = loss_1([a_0,b_0,c_0,d_0, e_0, f_0], coord_ab, coord_cd, coord_ef)
        min_loss_2 = loss_2([a_0,b_0,c_0,d_0, e_0, f_0], coord_ab, coord_cd, coord_ef) 
        
        # Init Loss
        println("Init Loss: ", min_loss)
        push!(loss_, min_loss)
    else
        println("Loss: ", min_loss, " ---- ", min_loss_2)
        push!(loss_, min_loss)
    end
    

    ####
    #We move one both points in each pair
    ####

    # See if stationary
    old_b = b_0
    old_d = d_0
    old_f = f_0
    old_a = a_0
    old_c = c_0
    old_e = e_0



    bol1 = true
    bol2 = true
    bol3 = true


    vec_0 = Array([data[b_0][1],data[b_0][2],data[b_0][3]])
    vec_1 = Array([data[d_0][1],data[d_0][2],data[d_0][3]])
    vec_2 = Array([data[f_0][1],data[f_0][2],data[f_0][3]])
    
    b_0_set, dists = knn(kdtree, vec_0,K, true)
    d_0_set, dists = knn(kdtree, vec_1,K, true)
    f_0_set, dists = knn(kdtree, vec_2,K, true)


    vec_0 = Array([data[a_0][1],data[a_0][2],data[a_0][3]])
    vec_1 = Array([data[c_0][1],data[c_0][2],data[c_0][3]])
    vec_2 = Array([data[e_0][1],data[e_0][2],data[e_0][3]])

    a_0_set, dists = knn(kdtree, vec_0,K, true)
    c_0_set, dists = knn(kdtree, vec_1,K, true)
    e_0_set, dists = knn(kdtree, vec_2,K, true)

    # find b_0^i and d_0^1 that minimizes loss



    a_0_i = a_0
    c_0_i = c_0
    e_0_i = e_0

    b_0_i = b_0
    d_0_i = d_0
    f_0_i = f_0
     
    if true#mod(k, 2) == 0
        if bol1
            for i in 1:length(b_0_set)
                b_0_i = b_0_set[i]
                l = loss_1([a_0,b_0_i,c_0,d_0, e_0, f_0], coord_ab, coord_cd, coord_ef)
                if  l < min_loss
                    min_loss = l
                    b_0 = b_0_i
                    bol1 = false
                end
            end
        end
        if bol2
            for i in 1:length(c_0_set)
                c_0_i  = c_0_set[i]
                l = loss_1([a_0,b_0,c_0_i,d_0, e_0, f_0], coord_ab, coord_cd, coord_ef)
                if  l < min_loss
                    min_loss = l
                    c_0 = c_0_i
                    bol2 = false
                end
            end
        end
    
        if bol2
            for i in 1:length(d_0_set)
                d_0_i  = d_0_set[i]
                l = loss_1([a_0,b_0,c_0,d_0_i, e_0, f_0], coord_ab, coord_cd, coord_ef)
                if  l < min_loss
                    min_loss = l
                    d_0 = d_0_i
                    #bol1 = false
                end
            end
        end
        if bol3
            for i in 1:length(e_0_set)
                e_0_i  = e_0_set[i]
                l = loss_1([a_0,b_0,c_0,d_0, e_0_i, f_0], coord_ab, coord_cd, coord_ef)
                if  l < min_loss
                    min_loss = l
                    e_0 = e_0_i
                    bol3 = false
                end
            end
        end
        if bol3
            for i in 1:length(f_0_set)
                f_0_i  = f_0_set[i]
                l = loss_1([a_0,b_0,c_0,d_0, e_0, f_0_i], coord_ab, coord_cd, coord_ef)
                if  l < min_loss
                    min_loss = l
                    f_0 = f_0_i
                end
            end
        end

    else
        println(" DOing the other loss function")
        if bol1
            for i in 1:length(b_0_set)
                b_0_i = b_0_set[i]
                l = loss_2([a_0,b_0_i,c_0,d_0, e_0, f_0], coord_ab, coord_cd, coord_ef)
                if  l < min_loss_2
                    min_loss_2 = l
                    b_0 = b_0_i
                    bol1 = false
                end
            end
        end
        if bol2
            for i in 1:length(c_0_set)
                c_0_i  = c_0_set[i]
                l = loss_2([a_0,b_0,c_0_i,d_0, e_0, f_0], coord_ab, coord_cd, coord_ef)
                if  l < min_loss_2
                    min_loss_2 = l
                    c_0 = c_0_i
                    bol2 = false
                end
            end
        end
    
        if bol2
            for i in 1:length(d_0_set)
                d_0_i  = d_0_set[i]
                l = loss_2([a_0,b_0,c_0,d_0_i, e_0, f_0], coord_ab, coord_cd, coord_ef)
                if  l < min_loss_2
                    min_loss_2 = l
                    d_0 = d_0_i
                    #bol1 = false
                end
            end
        end
        if bol3
            for i in 1:length(e_0_set)
                e_0_i  = e_0_set[i]
                l = loss_2([a_0,b_0,c_0,d_0, e_0_i, f_0], coord_ab, coord_cd, coord_ef)
                if  l < min_loss_2
                    min_loss_2 = l
                    e_0 = e_0_i
                    bol3 = false
                end
            end
        end
        if bol3
            for i in 1:length(f_0_set)
                f_0_i  = f_0_set[i]
                l = loss_2([a_0,b_0,c_0,d_0, e_0, f_0_i], coord_ab, coord_cd, coord_ef)
                if  l < min_loss_2
                    min_loss_2 = l
                    f_0 = f_0_i
                end
            end
        end


    end
    #println("LOSS 2 ", min_loss_2)
 

    if old_b == b_0 && old_d == d_0 && old_f == f_0 && old_a == a_0 && old_c == c_0 && old_e == e_0
        print("Fix Point")
        break_count +=1
        if break_count == 2
            break
        end
        #break
        #K += 10
        #println("K = ", K)
    else
        break_count = 0
    end
    push!(a_, a_0)
    push!(b_, b_0)
    push!(c_, c_0)
    push!(d_, d_0)
    push!(e_, e_0)
    push!(f_, f_0)



    println("Done with: ", k)

end

Number of points: 100 
Careful if this number is too high: 4519
number of 1-simps: 

321
 
number of all simps: 1118
Starting second penetration


number of 1-simps: 

321
 
number of all simps: 1118
Starting third penetration


number of 1-simps: 

...

All penetration done.
points
[2,94,40,54,54,88]
-------
Loss: -0.18632751903198536 ---- 4.14024831578749
Done with: 2
Number of points: 100 
Careful if this number is too high: 4519
number of 1-simps: 

321
 
number of all simps: 1118
Starting second penetration


number of 1-simps: 

321
 
number of all simps: 1118
Starting third penetration


number of 1-simps: 

In [ ]:
loss_

In [ ]:
print("[" * string(a_[1]-1) * "," * string(b_[1]-1) * ","
 * string(c_[1]-1) * "," * string(d_[1]-1) *  "," * string(e_[1]-1) *  "," * string(f_[1]-1) * "]")

In [ ]:
l = length(a_)

print("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," 
* string(d_[l]-1) * "," * string(e_[l]-1) *  "," * string(f_[l]-1) * "]")